In [2]:
using Flux
using Flux: onehotbatch, argmax, crossentropy, throttle, @epochs
using Base.Iterators: repeated, partition
using MLDatasets

┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1242


In [4]:
BATCH_SIZE = 100

# load CIFAR-10 training set
trainX, trainY = CIFAR10.traindata()
testX,  testY  = CIFAR10.testdata()

# MLDatasets returns UInt8 thus convert it to Float64
trainX = Array{Float64}(trainX)
testX = Array{Float64}(testX)
println("conversion is done")

# construct one-hot vectors from labels
trainY = onehotbatch(trainY, 0:9)
testY = onehotbatch(testY, 0:9)

train = (trainX, trainY)

conversion is done


([0.23137254901960785 0.06274509803921569 … 0.7058823529411764 0.6941176470588235; 0.16862745098039217 0.0 … 0.6784313725490196 0.6588235294117647; … ; 0.596078431372549 0.4666666666666667 … 0.3803921568627451 0.592156862745098; 0.580392156862745 0.4784313725490196 … 0.3254901960784314 0.4823529411764706]

[0.24313725490196078 0.0784313725490196 … 0.5450980392156862 0.5647058823529412; 0.1803921568627451 0.0 … 0.4823529411764706 0.5058823529411764; … ; 0.49019607843137253 0.3254901960784314 … 0.24313725490196078 0.4627450980392157; 0.48627450980392156 0.3411764705882353 … 0.20784313725490194 0.3607843137254902]

[0.24705882352941178 0.0784313725490196 … 0.3764705882352941 0.4549019607843137; 0.1764705882352941 0.0 … 0.16470588235294117 0.3686274509803922; … ; 0.4 0.19607843137254902 … 0.13333333333333333 0.32941176470588235; 0.403921568627451 0.22352941176470587 … 0.13333333333333333 0.2823529411764706]

[0.6039215686274509 0.5490196078431373 … 0.6470588235294118 0.6392156862745098; 0.

In [3]:
# TODO convert below to list comprehension
# TODO shuffle
# split training set into batches
# train_data contains whole data in batches
train_data = Array{Any}(div(50000, BATCH_SIZE))
for i = 0:div(50000, BATCH_SIZE) - 1
    train_data[i+1] = train[1][:,:,:, 1 + i*BATCH_SIZE:(i+1)*BATCH_SIZE],
                      train[2][:, 1 + i*BATCH_SIZE:(i+1)*BATCH_SIZE]
end

println("data is ready to be learnt")

This program has requested access to the data dependency CIFAR10.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: The CIFAR-10 dataset
Authors: Alex Krizhevsky, Vinod Nair, Geoffrey Hinton
Website: https://www.cs.toronto.edu/~kriz/cifar.html
Reference: https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf

[Krizhevsky, 2009]
    Alex Krizhevsky.
    "Learning Multiple Layers of Features from Tiny Images",
    Tech Report, 2009.

The CIFAR-10 dataset is a labeled subsets of the 80
million tiny images dataset. It consists of 60000
32x32 colour images in 10 classes, with 6000 images
per class.

The compressed archive file that contains the
complete dataset is available for download at the
offical website linked above; specifically the binary
version for C programs. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of CIFAR-10 aren't really
explicit about an

MethodError: MethodError: no method matching Array{Any,N} where N(::Int64)
Closest candidates are:
  Array{Any,N} where N(!Matched::UndefInitializer, !Matched::Int64) where T at boot.jl:418
  Array{Any,N} where N(!Matched::UndefInitializer, !Matched::Int64, !Matched::Int64) where T at boot.jl:419
  Array{Any,N} where N(!Matched::UndefInitializer, !Matched::Int64, !Matched::Int64, !Matched::Int64) where T at boot.jl:420
  ...

In [ ]:
m = Chain(
  Conv((3,3), 3=>16, relu),
  x -> maxpool(x, (2,2)),
  Conv((2,2), 16=>8, relu),
  x -> maxpool(x, (2,2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(8*7*7 , 10), softmax)

m(train_data[1][1])
loss(x, y) = crossentropy(m(x), y)
accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))
evalcb = throttle(() -> @show(accuracy(testX, testY)), 10)
opt = ADAM(params(m))

In [ ]:
# run this cell to train

# Flux.train!() runs for 1 epoch, default. 
# Change 15 to train for different epochs using @epochs macro
@epochs 15 Flux.train!(loss, train, opt, cb = evalcb)